# 03_07: Populating a Knowledge Graph into Neo4j using LangChain

In [1]:
!pip install openai
!pip install langchain
!pip install langchain-community
!pip install langchain_neo4j
!pip install langchain_openai
!pip install langchain_experimental


[notice] A new release of pip is available: 24.3.1 -> 25.0.1
[notice] To update, run: python3 -m pip install --upgrade pip

[notice] A new release of pip is available: 24.3.1 -> 25.0.1
[notice] To update, run: python3 -m pip install --upgrade pip

[notice] A new release of pip is available: 24.3.1 -> 25.0.1
[notice] To update, run: python3 -m pip install --upgrade pip

[notice] A new release of pip is available: 24.3.1 -> 25.0.1
[notice] To update, run: python3 -m pip install --upgrade pip

[notice] A new release of pip is available: 24.3.1 -> 25.0.1
[notice] To update, run: python3 -m pip install --upgrade pip

[notice] A new release of pip is available: 24.3.1 -> 25.0.1
[notice] To update, run: python3 -m pip install --upgrade pip


In [12]:
URI = "..."
USER = "neo4j"
PWD = "..."
AUTH = (USER, PWD)
OPENAI_API_KEY = "..."

In [5]:
import os

from langchain_neo4j import Neo4jGraph
from langchain_core.documents import Document
from langchain_community.graphs.graph_document import GraphDocument
from langchain_experimental.graph_transformers import LLMGraphTransformer
from langchain_openai import ChatOpenAI

graph = Neo4jGraph(url=URI, username=USER, password=PWD)

In [4]:
text = """
Ada Lovelace was a mathematician who worked with Charles Babbage on the Analytical Engine.
She is often credited as the first computer programmer.
"""

In [6]:
llm = ChatOpenAI(openai_api_key=OPENAI_API_KEY, temperature=0, model_name="gpt-4o")

llm_transformer = LLMGraphTransformer(llm=llm)

In [7]:
documents = [Document(page_content=text)]
graph_documents = llm_transformer.convert_to_graph_documents(documents)
print(f"Nodes:{graph_documents[0].nodes}")
print(f"Relationships:{graph_documents[0].relationships}")

Nodes:[Node(id='Ada Lovelace', type='Person', properties={}), Node(id='Charles Babbage', type='Person', properties={}), Node(id='Analytical Engine', type='Concept', properties={})]
Relationships:[Relationship(source=Node(id='Ada Lovelace', type='Person', properties={}), target=Node(id='Charles Babbage', type='Person', properties={}), type='WORKED_WITH', properties={}), Relationship(source=Node(id='Ada Lovelace', type='Person', properties={}), target=Node(id='Analytical Engine', type='Concept', properties={}), type='WORKED_ON', properties={}), Relationship(source=Node(id='Ada Lovelace', type='Person', properties={}), target=Node(id='First Computer Programmer', type='Concept', properties={}), type='CREDITED_AS', properties={})]


In [8]:
graph.add_graph_documents(graph_documents)

In [9]:
graph.query('MATCH (n) RETURN COUNT(n) as TOTAL')

[{'TOTAL': 4}]

In [10]:
graph.query('MATCH (n) DETACH DELETE n')

[]

In [11]:
graph.add_graph_documents(graph_documents, include_source=True)